# __Building and Visualizing an Autoencoder with the Fashion MNIST Dataset__

Autoencoders are a special type of neural network used for:

- Data compression
- Feature extraction
- Dimensionality reduction
- Learning generative models of data


## Steps to be followed
1. Import the libraries
2. Load the dataset and find the shape of the data
3. Initialize the autoencoder
4. Compile the autoencoder
5. Train the model
6. Visualize the images

###Step 1: Import the libraries
- Import the required libraries, such as NumPy, Pandas, TensorFlow, and Matplotlib.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

2024-11-05 07:16:17.233190: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 07:16:19.236901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 07:16:19.886265: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 07:16:20.068097: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 07:16:21.468258: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

### Step 2: Load the dataset and find the shape of the data

- Dataset used: Fashion MNIST dataset, where each image is 28 *28 pixels
- Find the shape of the train and test data.

In [4]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print (x_train.shape)
print (x_test.shape)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)


__Observation:__
- Here, the shape function retrieves the number of rows and columns present in the train and test data.

### Step 3: Initialize the autoencoder
- Define the value of __latent_dim__ as __64__.
- Define a class called __Autoencoder__ that extends the __Model__ class from TensorFlow.
- Inside the Autoencoder class, define the constructor (__init__) that takes __latent_dim__ as a parameter.
- In the constructor, set __self.latent_dim__ to the value of __latent_dim__.
- Define the encoder part of the autoencoder using __tf.keras.Sequential__.
- In the encoder, flatten the input using __layers.Flatten()__.
- Add a dense layer to the encoder with latent_dim units and ReLU activation using __layers.Dense(latent_dim, activation='relu')__.
- Define the decoder part of the autoencoder using __tf.keras.Sequential__.
- In the decoder, add a dense layer with __784__ units and sigmoid activation using __layers.Dense(784, activation='sigmoid')__.
- Reshape the output of the dense layer to a 28x28 shape using __layers.Reshape((28, 28))__.
- Define the call method of the __Autoencoder__ class.
- Inside the call method, pass the input x through the encoder to obtain the encoded representation.
- Pass the encoded representation through the decoder to obtain the reconstructed output.
- Return the reconstructed output.
- Create an instance of the __Autoencoder__ class called autoencoder, passing the value of __latent_dim__ as an argument.

In [5]:
latent_dim = 64

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28, 28))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

__Observations:__
- The code does not produce any output by itself. It defines a class and creates an instance of that class.
- The output will depend on how the autoencoder model is trained and used further in the code.

### Step 4: Compile the autoencoder
- Call the __compile()__ method on the autoencoder object.
- Set the optimizer argument to __adam__. This specifies that the Adam optimizer will be used for training the autoencoder.
- Set the loss argument to __losses.MeanSquaredError()__. This specifies that the mean squared error loss function will be used for training the autoencoder.

In [6]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

2024-11-05 07:16:45.143826: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


__Observation:__
- It configures the autoencoder model for training by setting the optimizer and loss functions.

### Step 5: Train the model
- Call the __fit()__ method on the autoencoder object.
- Pass __x_train__ as the first argument. x_train represents the input data for training the autoencoder.
- Pass __x_train__ again as the second argument. This is the target data for the autoencoder, which is also x_train in this case.
- Set the __epochs__ argument to __10__. This specifies the number of times the entire dataset will be iterated during training.
- Set the __shuffle__ argument to __True__. This indicates that the training data will be shuffled before each epoch during training.
- Set the validation_data argument to __(x_test, x_test)__. This provides the validation data needed to evaluate the performance of the autoencoder during training. x_test is the input validation data, and x_test is also used as the target validation data.

In [7]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0403 - val_loss: 0.0135
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0124 - val_loss: 0.0109
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0105 - val_loss: 0.0100
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0097 - val_loss: 0.0096
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0094 - val_loss: 0.0094
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0092 - val_loss: 0.0093
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0091 - val_loss: 0.0092
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0090 - val_loss: 0.0091
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0089 - val_loss: 0.0090


__Observations:__
- The __fit()__ method trains the autoencoder model on the provided data and produces output.
- During training, it displays information such as the loss and metrics for each epoch, the progress bar, and validation metrics if validation data is provided.
- The final output is the trained autoencoder model with updated weights.

### Step 6: Encode and decode the images
- Call the encoder method of the autoencoder object on __x_test__. This encodes the input x_test using the trained autoencoder's encoder part.
- Call the __numpy()__ method on the encoded output to convert it into a NumPy array.
- This is done to extract the actual values from the TensorFlow tensor.
- Assign the encoded output to the variable __encoded_imgs__.
- Call the decoder method of the autoencoder object on encoded_imgs. This decodes the encoded images using the trained autoencoder's decoder part.
- Call the numpy() method on the decoded output to convert it into a NumPy array.
- Assign the decoded output to the variable __decoded_imgs__.

In [8]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

### Step 7: Display the images

- Set up the figure and subplot layout.
- Iterate through a range of n (in this case, 10) for displaying original and reconstructed images.
- Display the original image in the current subplot, along with the __original__ title and grayscale colormap.
- Display the reconstructed image in the next subplot, along with the __reconstructed__ title and grayscale colormap.

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):

  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

__Observations:__
- The code generates a figure that showcases original images alongside their corresponding reconstructed images, with the __original__ and __reconstructed__ titles.
- The images are displayed in grayscale.